In [18]:
! pip -q install pandas plotly
! pip -q install nbformat

In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import glob
import json
from plotly.subplots import make_subplots
import math
import os

In [20]:
result_files = glob.glob("eval_results/fineweb/*/*_iter_*/*.json")
#baseline_hptl2_dedup = glob.glob("/scratch/project_462000353/pyysalos/second-hplt-eval/eval_results/results/hplt_v2_dedup_iter_00*/*.json")
#baseline_fineweb = glob.glob("/scratch/project_462000353/pyysalos/second-hplt-eval/eval_results/results/fineweb_iter_00*/*.json")

tasks = {"custom|arc:easy|0":"arc_easy",
         "custom|arc:challenge|0":"arc_challenge",
         "custom|hellaswag|0":"hellaswag",
         "custom|commonsense_qa|0": "commonsense_qa",
         "custom|openbookqa|0": "openbookqa",
         "custom|piqa|0": "piqa",
         "custom|siqa|0": "siqa",
         "custom|mmlu:_average|0": "mmlu",
         "custom|winogrande|0": "winogrande",
}



In [22]:
def extract_results(result_files, recalc_mean=False):
    data = []
    for f in result_files:
        splitted = f.split("/")
        #print(splitted)
        register = splitted[-3] #splitted[-2].split("_iter_")[0]
        step = splitted[-2].split("_")[-1] #splitted[-2].replace(f'{register}_iter_', "")
        step=int(step)
        #print(register)
        #print(step)
        with open(f, 'r') as result:
            j = json.load(result)
            results = {"step":int(step)}
            if step > 50000:
                continue
            results_all = j["results"]["all"]
            results = {**results, **results_all}
            for task, task_name in tasks.items():
                r = {k+f"_{task_name}":v for k,v in j["results"][task].items()}
                results = {**results, **r}
            #print(results)
            df = pd.DataFrame.from_dict({register:{"step":int(step), **results}})
            data.append(df.T)
            #print(register, j["results"]["all"])
            #data[register] = {step: j["results"]["all"]}

    df = pd.concat(data)
    df = df.reset_index()
    df = df.rename(columns={"index":"register"})
    # problem: step does not go to correct place on the plot
    # sort it as numerical, then make it categorical so plotting works
    # ordered=True does nothing here because plotly does not respect it; only sorting works
    df=df.sort_values("step")
    df["tokens"] = df["step"]*2.1e6
    #df["step"] = pd.Categorical(df['step'], categories=sorted(df['step'].astype(int).unique()), ordered=True)
    df["step"]= df['step'].apply(lambda x: str(round(x)))
    #df["tokens"]= df['tokens'].apply(lambda x: millify(x))
    if recalc_mean:
        df["acc_norm_original"] = df["acc_norm"]
        #df["acc_norm"] = df[["acc_norm_hellaswag", "acc_norm_openbookqa", "acc_norm_piqa", "acc_norm_arc_easy"]].mean(axis=1)
        df["acc_norm"] = df[[f"acc_norm_{taskname}" for taskname in tasks.values()]].mean(axis=1)
        df["stderr_norm"] = df[[f"acc_norm_stderr_{taskname}" for taskname in tasks.values()]].apply(lambda x: x**2).sum(axis=1)
        df["stderr_norm"] = df["stderr_norm"].apply(lambda x: math.sqrt(x))
    return df

df1 = extract_results(result_files, recalc_mean=True)
#df2 = extract_results(baseline_hptl2_dedup, recalc_mean=True)
#df3 = extract_results(baseline_fineweb, recalc_mean=True)

In [23]:
df = df1 #pd.concat([df1, df2, df3])
display(df)

print(df.columns)
# another way to calculate this, just to check
#df["acc_norm_corr"] = df[[f'acc_norm_'+i for i in tasks.values() if i != "acc_norm"]].mean(axis=1)
#display(df)
#print(df["acc_norm_corr"].equals(df["acc_norm"]))

,register,acc,acc_arc_challenge,acc_arc_easy,acc_commonsense_qa,acc_hellaswag,acc_mmlu,acc_norm,acc_norm_arc_challenge,acc_norm_arc_easy,...,acc_stderr_mmlu,acc_stderr_openbookqa,acc_stderr_piqa,acc_stderr_siqa,acc_stderr_winogrande,acc_winogrande,step,tokens,acc_norm_original,stderr_norm
664,IP,0.244810,0.169,0.283,0.214,0.284,0.234204,0.336226,0.214,0.319,...,0.031430,0.016158,0.015626,0.015258,0.015814,0.513,1000,2.100000e+09,0.261104,0.054051
231,LY,0.241178,0.170,0.304,0.229,0.273,0.231080,0.322654,0.226,0.300,...,0.031278,0.014651,0.015789,0.015091,0.015798,0.526,1000,2.100000e+09,0.261681,0.053812
300,fineweb,0.251400,0.183,0.375,0.234,0.276,0.239561,0.348450,0.223,0.354,...,0.031594,0.015055,0.015525,0.015317,0.015810,0.517,1000,2.100000e+09,0.269707,0.054510
199,HI,0.245950,0.171,0.307,0.252,0.296,0.233294,0.351006,0.222,0.315,...,0.031361,0.016158,0.015071,0.015196,0.015819,0.496,1000,2.100000e+09,0.264036,0.054150
526,HI-IN_HI_dtp_NA_OP,0.251779,0.155,0.365,0.255,0.291,0.239485,0.354445,0.216,0.357,...,0.031481,0.015440,0.015168,0.015168,0.015815,0.511,1000,2.100000e+09,0.264463,0.054272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,MT,0.266484,0.197,0.458,0.243,0.324,0.253306,0.374349,0.229,0.433,...,0.032267,0.014956,0.015214,0.015232,0.015789,0.531,50000,1.050000e+11,0.281125,0.055054
9,dtp,0.309085,0.282,0.643,0.357,0.405,0.290220,0.452347,0.303,0.606,...,0.033388,0.018781,0.014633,0.015473,0.015749,0.547,50000,1.050000e+11,0.329816,0.057087
178,HI,0.276587,0.235,0.489,0.345,0.478,0.254809,0.446558,0.281,0.478,...,0.032176,0.017973,0.013191,0.015357,0.015743,0.549,50000,1.050000e+11,0.294467,0.055678
63,SP,0.289890,0.226,0.484,0.390,0.395,0.272822,0.422314,0.276,0.461,...,0.032938,0.017048,0.014911,0.015525,0.015740,0.550,50000,1.050000e+11,0.303859,0.056512


Index(['register', 'acc', 'acc_arc_challenge', 'acc_arc_easy',
       'acc_commonsense_qa', 'acc_hellaswag', 'acc_mmlu', 'acc_norm',
       'acc_norm_arc_challenge', 'acc_norm_arc_easy',
       'acc_norm_commonsense_qa', 'acc_norm_hellaswag', 'acc_norm_mmlu',
       'acc_norm_openbookqa', 'acc_norm_piqa', 'acc_norm_siqa',
       'acc_norm_stderr', 'acc_norm_stderr_arc_challenge',
       'acc_norm_stderr_arc_easy', 'acc_norm_stderr_commonsense_qa',
       'acc_norm_stderr_hellaswag', 'acc_norm_stderr_mmlu',
       'acc_norm_stderr_openbookqa', 'acc_norm_stderr_piqa',
       'acc_norm_stderr_siqa', 'acc_norm_stderr_winogrande',
       'acc_norm_winogrande', 'acc_openbookqa', 'acc_piqa', 'acc_siqa',
       'acc_stderr', 'acc_stderr_arc_challenge', 'acc_stderr_arc_easy',
       'acc_stderr_commonsense_qa', 'acc_stderr_hellaswag', 'acc_stderr_mmlu',
       'acc_stderr_openbookqa', 'acc_stderr_piqa', 'acc_stderr_siqa',
       'acc_stderr_winogrande', 'acc_winogrande', 'step', 'tokens',
     

In [ ]:
fig = px.line(df, y="acc_norm", x="tokens", color="register", color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.update_layout(
    title="Average accuracy by Register",
    height=600,  # Adjust the height of the plot
    width=850,
    showlegend=True,  # Show legend
)
#fig.update_traces(connectgaps=False)
fig.update_layout(
    template='simple_white',
    #plot_bgcolor='rgba(0, 0, 0, 0)',
    #paper_bgcolor='rgba(0, 0, 0, 0)',
)
fig.show()

In [ ]:
unique_registers = df["register"].unique()
#colors = px.colors.sample_colorscale("mygbm",len(unique_registers))  # for other colormaps
c_map={}
for i, register in enumerate(unique_registers):
    c_map[register] = px.colors.qualitative.Alphabet[i]   # this is the same as above
print(c_map)

{'HI-IN_HI_dtp_NA_OP': '#AA0DFE', 'LY': '#3283FE', 'dtp': '#85660D', 'IN': '#782AB6', 'ID': '#565656', 'NA': '#1C8356', 'fineweb': '#16FF32', 'IP': '#F7E1A0', 'SP': '#E2E2E2', 'HI-IN': '#1CBE4F', 'HI': '#C4451C', 'OP': '#DEA0FD', 'ne': '#FE00FA', 'hplt-v2-dedup': '#325A9B', 'MT': '#FEAF16', 'HI-IN_HI_dtp_OP': '#F8A19F', 'HI-IN_HI_dtp': '#90AD1C'}


In [ ]:

# Create a 2x2 grid of subplots
fig = make_subplots(
    rows=3, cols=3, 
    subplot_titles=["arc:easy", "hellaswag", "mmlu", "arc:challenge", "siqa", "piqa", "winogrande", "commonsenseqa", "openbookqa"],
    x_title="Tokens",
    y_title="Accuracy score",
    #shared_xaxes=True,  # Share the x-axis across all subplots
    vertical_spacing=0.09,  # Adjust space between subplots
)

# List of accuracy columns
accuracy_columns = ['acc_norm_arc_easy', 'acc_norm_hellaswag', 'acc_norm_mmlu', 'acc_norm_arc_challenge', 'acc_norm_siqa', 'acc_norm_piqa', 'acc_norm_winogrande', 'acc_norm_commonsense_qa', 'acc_norm_openbookqa']

# Loop over each accuracy metric and add traces to the subplots
for i, accuracy_col in enumerate(accuracy_columns):
    row = (i // 3) + 1  # Determine the row for the subplot (1 or 2)
    col = (i % 3) + 1   # Determine the column for the subplot (1 or 2)
    
    # Group by label and plot each group with a different color
    for label in df['register'].unique():
        label_data = df[df['register'] == label]
        
        fig.add_trace(
            go.Scatter(
                x=label_data['tokens'], 
                y=label_data[accuracy_col], 
                mode='lines', 
                showlegend=True if row==1 and col==1 else False,
                name=f'{label}', #if row==0 and col==0 else None,#name=f'{accuracy_col} - {label}', 
                legendgroup=f'{label}',
                line=dict(color=c_map[label])#dict(color=px.colors.qualitative.Set1[list(df['register'].unique()).index(label)])
            ),
            row=row, col=col
        )

# Update layout for better appearance
fig.update_layout(
    title="Accuracy Metrics by Task and Register",
    height=1000,  # Adjust the height of the plot
    width=1300,
    showlegend=True,  # Show legend
)


# Show the plot
fig.show()